# 7-2. Pandas와 데이터베이스 연동
SqlAlchemy 패키지를 활용하여 Pandas의 DataFrame과 데이터베이스를 직접 연동시키는 방법

# #01. 패키지 설치

In [1]:
pip install --upgrade sqlalchemy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# #02. 연동준비
## 1) 패키지 참조

In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from pandas import DataFrame
from matplotlib import pyplot

## 2) 데이터베이스 연동에 필요한 접속 정보

In [3]:
HOSTNAME = 'localhost'
PORT = 3306
USERNAME = 'root'
PASSWORD = '123qwe!@#'
DATABASE = 'myschool'
CHARSET1 = 'utf8'          # MySQL에서 사용할 캐릭터셋
CHARSET2 = 'utf-8'         # Python에서 사용할 캐릭터셋

# #03. 데이터베이스 접속하기
## 1) 접속 문자열 생성
mysql+mysqldb://아이디:비밀번호@DB서버주소:포트번호/DB이름?charset=인코딩

In [4]:
con_str_fmt = "mysql+mysqldb://{0}:{1}@{2}:{3}/{4}?charset={5}"
con_str = con_str_fmt.format(USERNAME, PASSWORD, HOSTNAME, PORT, DATABASE, CHARSET1)
con_str

'mysql+mysqldb://root:123qwe!@#@localhost:3306/myschool?charset=utf8'

## 2) pymysql을 사용하여 MySQL 연동 객체 설치하기

In [5]:
pymysql.install_as_MySQLdb()
import MySQLdb

## 3) 데이터베이스 접속

In [7]:
engine = create_engine(con_str, encoding=CHARSET2)
conn = engine.connect()

# #05. 데이저 저장하기
## 1) 인덱스 컬럼이 없는 DataFrame의 경우
### DataFrame 생성
엑셀읽기, 크롤링 등의 방법으로 수집한 데이터를 DataFrame으로 생성한다.

In [8]:
df1 = DataFrame([
    {'deptno':300, 'dname':'학과1', 'loc':'위치1'},
    {'deptno':301, 'dname':'학과2', 'loc':'위치2'},
    {'deptno':302, 'dname':'학과3', 'loc':'위치3'},
])
df1

,deptno,dname,loc
0,300,학과1,위치1
1,301,학과2,위치2
2,302,학과3,위치3


### DataFrame을 db에 저장하기
`to_sql()` 함수는 지정된 테이블이 존재하지 않을 경우 새로 생성하여 데이터를 저장한다.

파라미터 | 설명 |
---- | ---- |
if_exists | replace : index값을 기준으로 동일한 데이터 수정 |
if_exists | append : 새로운 행으로 추가 |
index | True : DataFrame의 index가 DB에 함께 저장 |
index | DataFrame의 index를PK로 사용할 경우만 적용할 것

In [9]:
# 인덱스와 함께 저장하기
df1.to_sql(name = 'department_py', con=conn, if_exists='replace', index=True)

In [10]:
# 인덱스를 제외하고 저장하기
df1.to_sql(name = 'department_py', con=conn, if_exists='replace', index=False)

## 2) 인덱스 컬럼이 있는 DataFrame의 경우
가장 이상적인 형태는 DataFrame에 별도의 인덱스 컬럼을 지정하고, DB에 저장할 때 그 컬럼이 함께 저장되도록 설정하는 것이다.

### DataFrame 만들기

In [12]:
df2 = DataFrame([
    {'deptno':300, 'dname':'학과1', 'loc':'위치1'},
    {'deptno':301, 'dname':'학과2', 'loc':'위치2'},
    {'deptno':302, 'dname':'학과3', 'loc':'위치3'}    
])
df2.set_index('deptno', inplace=True)
df2

,dname,loc
deptno,,
300,학과1,위치1
301,학과2,위치2
302,학과3,위치3


### 데이터 저장하기

In [13]:
df2.to_sql(name='department_py', con=conn, if_exists='replace', index=True)

# #05. 데이터 조회하기
## 1) 부분조회
원하는 컬럼 이름을 나열하고 WHERE절로 특정 조건에 맞는 데이터 조회하기

In [14]:
sql = "SELECT deptno, dname, loc FROM department_py WHERE deptno=300"
df3 = pd.read_sql(sql, index_col='deptno', con=conn)
df3

,dname,loc
deptno,,
300,학과1,위치1


## 2) 전체조회
### 특정 테이블 데이터를 통째로 다 가져오기

In [17]:
df4 = pd.read_sql_table('department_py', index_col='deptno', con=conn)
df4

,dname,loc
deptno,,
300,학과1,위치1
301,학과2,위치2
302,학과3,위치3


### 가져올 컬럼 지정하기

In [18]:
df5 = pd.read_sql_table('department_py',
                       index_col = 'deptno', 
                       columns=['dname'],
                       con=conn)
df5

,dname
deptno,
300,학과1
301,학과2
302,학과3


# #06. 데이터베이스 접속 해제

In [19]:
conn.close()